<코드참조>
https://www.tensorflow.org/recommenders/examples/dcn

https://www.tensorflow.org/datasets/catalog/movie_lens

In [ ]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

In [ ]:
import pprint

%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

import tensorflow_recommenders as tfrs

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 램초과
wine_raw = '/content/drive/MyDrive/tobigs14_conference/data/v_2/wine_meta/train_all_meta_v2.json'
wine_raw = pd.read_json(wine_raw)
wine_raw

In [ ]:
wine_meta = '/content/drive/MyDrive/tobigs14_conference/data/v_2/wine_meta/Wine_Meta_final_201210.json'
wine_meta = pd.read_json(wine_meta)
wine_meta

In [ ]:
wine_meta_rank=wine_meta[['wine_id','world_rank', 'type_id', 'country_code']]

# train set 만들기

In [ ]:
user_train = '/content/drive/MyDrive/tobigs14_conference/data/v_2/userdata/user_train.json'
user_train = pd.read_json(user_train)
user_train

In [ ]:
user_train=pd.merge(user_train, wine_meta_rank)

In [ ]:
user_train.head()

In [ ]:
user_train={"wine_id": user_train["wine_id"],
    "userID": user_train["userID"],
    "rating_per_user": user_train["rating_per_user"],
    "user_like_count": user_train["user_like_count"],
    "type_id": user_train["type_id"],
    "world_rank": user_train["world_rank"]}

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices(user_train)

In [ ]:
train_dataset

In [ ]:
user_train_1 = train_dataset.map(lambda x: {
    "wine_id": x["wine_id"],
    "userID": x["userID"],
    "rating_per_user": x["rating_per_user"],
    "type_id": x["type_id"],
    "user_like_count": x["user_like_count"]
})

In [ ]:
user_train_1

In [ ]:
tf.random.set_seed(42)
shuffled = user_train_1.shuffle(937756, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(937756)

In [ ]:
feature_names = ["wine_id", "userID", "rating_per_user", "type_id", "user_like_count"]

vocabularies = {}

for feature_name in feature_names:
  vocab = user_train_1.batch(937756).map(lambda x: x[feature_name])
  vocabularies[feature_name] = np.unique(np.concatenate(list(vocab)))

In [ ]:
user_train_1

# testset 만들기

In [ ]:
user_test = '/content/drive/MyDrive/tobigs14_conference/data/v_2/userdata/user_test.json'
user_test = pd.read_json(user_test)
user_test

In [ ]:
user_test=pd.merge(user_test, wine_meta_rank)

In [ ]:
user_test={"wine_id": user_test["wine_id"],
    "userID": user_test["userID"],
    "rating_per_user": user_test["rating_per_user"],
    "type_id": user_test["type_id"],
    "user_like_count": user_test["user_like_count"]}

In [ ]:
test_dataset = tf.data.Dataset.from_tensor_slices(user_test)

In [ ]:
test_dataset

In [ ]:
user_test_1 = test_dataset.map(lambda x: {
    "wine_id": x["wine_id"],
    "userID": x["userID"],
    "rating_per_user": x["rating_per_user"],
    "type_id": x["type_id"],
    "user_like_count": x["user_like_count"]
})

In [ ]:
user_test_1

In [ ]:
tf.random.set_seed(42)
shuffled = user_test_1.shuffle(6343 , seed=42, reshuffle_each_iteration=False)

test = shuffled.take(6343)

In [ ]:
user_test_1

# 모델 트레이닝

In [ ]:
class DCN(tfrs.Model):

  def __init__(self, use_cross_layer, deep_layer_sizes, projection_dim=None):
    super().__init__()

    self.embedding_dimension = 32

    str_features = []
    int_features = ["userID", "type_id","user_like_count"]

    self._all_features = str_features + int_features
    self._embeddings = {}

    # Compute embeddings for string features.
    for feature_name in str_features:
      vocabulary = vocabularies[feature_name]
      self._embeddings[feature_name] = tf.keras.Sequential(
          [tf.keras.layers.experimental.preprocessing.StringLookup(
              vocabulary=vocabulary, mask_token=None),
           tf.keras.layers.Embedding(len(vocabulary) + 1,
                                     self.embedding_dimension)
    ])
      
    # Compute embeddings for int features.
    for feature_name in int_features:
      vocabulary = vocabularies[feature_name]
      self._embeddings[feature_name] = tf.keras.Sequential(
          [tf.keras.layers.experimental.preprocessing.IntegerLookup(
              vocabulary=vocabulary, mask_value=None),
           tf.keras.layers.Embedding(len(vocabulary) + 1,
                                     self.embedding_dimension)
    ])

    if use_cross_layer:
      self._cross_layer = tfrs.layers.dcn.Cross(
          projection_dim=projection_dim,
          kernel_initializer="glorot_uniform")
    else:
      self._cross_layer = None

    self._deep_layers = [tf.keras.layers.Dense(layer_size, activation="relu")
      for layer_size in deep_layer_sizes]

    self._logit_layer = tf.keras.layers.Dense(1)

    self.task = tfrs.tasks.Ranking(
      loss=tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.RootMeanSquaredError("RMSE")]
    )

  def call(self, features):
    # Concatenate embeddings
    embeddings = []
    for feature_name in self._all_features:
      embedding_fn = self._embeddings[feature_name]
      embeddings.append(embedding_fn(features[feature_name]))
    
    x = tf.concat(embeddings, axis=1)

    # Build Cross Network
    if self._cross_layer is not None:
      x = self._cross_layer(x)
    
    # Build Deep Network
    for deep_layer in self._deep_layers:
      x = deep_layer(x)

    return self._logit_layer(x)

  def compute_loss(self, features, training=False):
    labels = features.pop("wine_id")
    scores = self(features)
    return self.task(
        labels=labels,
        predictions=scores,
    )

In [ ]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [ ]:
def run_models(use_cross_layer, deep_layer_sizes, projection_dim=None, num_runs=5):
  models = []
  rmses = []

  for i in range(num_runs):
    model = DCN(use_cross_layer=use_cross_layer,
                deep_layer_sizes=deep_layer_sizes,
                projection_dim=projection_dim)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate))
    models.append(model)

    model.fit(cached_train, epochs=epochs, verbose=False)
    metrics = model.evaluate(cached_test, return_dict=True)
    rmses.append(metrics["RMSE"])

  mean, stdv = np.average(rmses), np.std(rmses)

  return {"model": models, "mean": mean, "stdv": stdv}

In [ ]:
epochs = 8
learning_rate = 0.01

In [ ]:
dcn_result = run_models(use_cross_layer=True,
                        deep_layer_sizes=[192, 192])

In [ ]:
dcn_lr_result = run_models(use_cross_layer=True,
                           projection_dim=3,
                           deep_layer_sizes=[192, 192])

In [ ]:
dnn_result = run_models(use_cross_layer=False,
                        deep_layer_sizes=[192, 192, 192])

In [ ]:
print("DCN            RMSE mean: {:.4f}, stdv: {:.4f}".format(
    dcn_result["mean"], dcn_result["stdv"]))
print("DCN (low-rank) RMSE mean: {:.4f}, stdv: {:.4f}".format(
    dcn_lr_result["mean"], dcn_lr_result["stdv"]))
print("DNN            RMSE mean: {:.4f}, stdv: {:.4f}".format(
    dnn_result["mean"], dnn_result["stdv"]))

In [ ]:
model = dcn_result["model"][0]
mat = model._cross_layer._dense.kernel
features = model._all_features

block_norm = np.ones([len(features), len(features)])

dim = model.embedding_dimension

# Compute the norms of the blocks.
for i in range(len(features)):
  for j in range(len(features)):
    block = mat[i * dim:(i + 1) * dim,
                j * dim:(j + 1) * dim]
    block_norm[i,j] = np.linalg.norm(block, ord="fro")

plt.figure(figsize=(9,9))
im = plt.matshow(block_norm, cmap=plt.cm.Blues)
ax = plt.gca()
divider = make_axes_locatable(plt.gca())
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im, cax=cax)
cax.tick_params(labelsize=10) 
_ = ax.set_xticklabels([""] + features, rotation=45, ha="left", fontsize=10)
_ = ax.set_yticklabels([""] + features, fontsize=10)

## 결과
- input에 텐서플로우에서는 string자료와 int자료를 넣어서 했지만, 여기서 string 자료를 넣을때는 data를 tensor화 하지 못했다. <br>
그리고 float형태의 rating per user를 input으로 넣으려 했으나 int값을 넣으라는 오류가 떴다. <br>
돌아가기는 하지만 형태만 갖춘 허접한 결과값을 얻었다. 다양한 데이터들을 넣어보면서 실험 해봐야할 것같다. 